In [1]:
import os
os.chdir('../')  # Moving up one directory to the root

In [2]:
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from main import SentimentAnalyzer
from models.sentiment_model import EnhancedDistilBertForSentiment, ModelTrainer
from config.model_config import ModelConfig
import matplotlib.pyplot as plt
import seaborn as sns

2024-11-30 09:11:08.769000: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-30 09:11:09.264017: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 09:11:09.264101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 09:11:09.349085: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 09:11:09.512298: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Initialize analyzer
analyzer = SentimentAnalyzer()

# Process and show preprocessing examples
train_texts, val_texts, train_labels, val_labels = analyzer.process_data()


Preprocessing Examples:

Original: Great service and amazing food!
Processed: Great service and amazing food!

Original: Terrible experience, would not recommend.
Processed: Terrible experience, would not_NEG recommend.

Original: The food was okay, but the service could be better.
Processed: The food was okay, but the service could be better.

Original: Yeah right, like that's going to work...
Processed: Yeah right, like that is going to work ELLIPSIS  _SARC_yeah right

Original: Thanks a lot... now everything is broken 🙄
Processed: Thanks a lot ELLIPSIS  now everything is broken 🙄 _SARC_thanks a lot (contextual)

Original: Obviously this is the best restaurant ever...
Processed: Obviously this is the best restaurant ever ELLIPSIS 


In [4]:
# Initialize model with your specific configuration
# Create the analyzer instance first
analyzer = SentimentAnalyzer()

# Then initialize the model
analyzer.initialize_model()
analyzer.initialize_model()

# Show model configuration
print("Model Configuration:")
print(f"BERT Model: {ModelConfig.BERT_MODEL}")
print(f"Learning Rate: {ModelConfig.LEARNING_RATE}")
print(f"Batch Size: {ModelConfig.BATCH_SIZE}")
print(f"Max Length: {ModelConfig.MAX_LENGTH}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-11-30 09:11:45.096179: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: UNKNOWN ERROR (34)
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identica

Model Configuration:
BERT Model: distilbert-base-uncased
Learning Rate: 1e-05
Batch Size: 32
Max Length: 200


In [5]:
# Create a dummy input to build the model
dummy_input = analyzer.tokenizer(
    "This is a dummy text", 
    return_tensors='tf',
    padding=True,
    truncation=True,
    max_length=ModelConfig.MAX_LENGTH
)

# Build the model by passing dummy input
_ = analyzer.model(dummy_input)  # Use underscore instead of asterisk
analyzer.model.summary()

Model: "enhanced_distil_bert_for_sentiment_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_distil_bert_model_1 (TF  multiple                  66362880  
 DistilBertModel)                                                
                                                                 
 bidirectional_1 (Bidirecti  multiple                  1154400   
 onal)                                                           
                                                                 
 dense_6 (Dense)             multiple                  30048     
                                                                 
 dense_7 (Dense)             multiple                  30048     
                                                                 
 dense_8 (Dense)             multiple                  30048     
                                                                 
 dense_9 (Dense)             m

In [ ]:
# Train model

history = analyzer.train()

# Training visualization - using actual metrics from your model
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()

Epoch 1/5


In [ ]:
test_cases = [
    "I do not like this restaurant",
    "The food wasn't very good",
    "I'll never come back here",
    "Yeah right, this is the best service ever...",
    "Thanks a lot for ruining my evening!!!",
    "Obviously this is the perfect meal... 🙄",
]

# Get predictions
for text in test_cases:
    results = analyzer.predict(text)
    print(f"\nText: {text}")
    print("Predictions:", {
        'negative': f"{results['negative']:.3f}",
        'neutral': f"{results['neutral']:.3f}",
        'positive': f"{results['positive']:.3f}",
        'has_sarcasm': results['has_sarcasm'],
        'has_negation': results['has_negation'],
        'is_multipolar': results['is_multipolar']
    })

In [ ]:
# Analyzing both sarcasm and negation detection
special_cases = [
    "This couldn't possibly be any better... 🙄",
    "Thank you so much for not helping at all!!!",
]

print("\nSpecial Feature Analysis:")
for text in special_cases:
    results = analyzer.predict(text)
    print(f"\nText: {text}")
    print(f"Sarcasm Detected: {results['has_sarcasm']}")
    print(f"Negation Detected: {results['has_negation']}")
    print(f"Multipolar: {results['is_multipolar']}")
    print(f"Sentiment Distribution: neg={results['negative']:.2f}, neu={results['neutral']:.2f}, pos={results['positive']:.2f}")